# XGBoost Model Training



In [1]:
# Setup Environment

import os
import logging
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import xgboost as xgb
import optuna
import itertools
from sklearn.preprocessing import LabelEncoder
import pyspark.sql.functions as F
from pyspark.sql.functions import (col, count, row_number, abs, unix_timestamp, mean, 
                                   when, lit, min as F_min, max as F_max , upper, trim,
                                   row_number, mean as F_mean, countDistinct, last, first, when)
from src.data_preprocessing.data_prep1.data_utils import initialize_environment 
# Set global references to None
spark = None
master_results_df = None
df = None
training_data = None
train_df = None

In [2]:

spark, jdbc_url, jdbc_properties, parquet_dir, log_file = initialize_environment()


25/01/13 16:38:52 WARN Utils: Your hostname, mail.relufox.ai resolves to a loopback address: 127.0.0.1; using 192.168.4.25 instead (on interface eno2)
25/01/13 16:38:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.4.4-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/exx/.ivy2/cache
The jars for the packages stored in: /home/exx/.ivy2/jars
ml.dmlc#xgboost4j-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f929444a-625e-45bb-9f34-e90e2f0cffd0;1.0
	confs: [default]
	found ml.dmlc#xgboost4j-spark_2.12;1.7.5 in central
	found ml.dmlc#xgboost4j_2.12;1.7.5 in central
	found com.typesafe.akka#akka-actor_2.12;2.5.23 in central
	found com.typesafe#config;1.3.3 in central
	found org.scala-lang.modules#scala-java8-compat_2.12;0.8.0 in central
	found com.esotericsoftware#kryo;4.0.2 in central
	found com.esotericsoftware#reflectasm;1.11.3 in central
	found org.ow2.asm#asm;5.0.4 in central
	found com.esotericsoftware#minlog;1.3.0 in central
	found org.objenesis#objenesis;2.5.1 in central
	found org.scala-lang#scala-reflect;2.12.8 in central
	found commons-logging#commons-logging;1.2 in central
:: resolution report :: resolve 136ms :: artifacts dl 5ms
	:: modules in use:
	com.esoter

Spark session created successfully.


In [3]:
training_data = spark.read.parquet("/home/exx/myCode/horse-racing/FoxRiverAIRacing/data/parquet/train_df")

In [4]:
training_data.count()

394322

# Switching to Pandas

In [5]:
# Convert Spark DataFrame -> Pandas DataFrame
training_data = training_data.toPandas()
# Quick info about the DataFrame
#print(df.info())
#print(df.head(5))

## Set the race_id

In [6]:
training_data["race_id"] = (
    training_data["course_cd"].astype(str) + "_" +
    training_data["race_date"].astype(str) + "_" +
    training_data["race_number"].astype(str)
)

## Group and sort data by race_id and group_id

In [7]:
# Generate unique numeric group_id from race_id
training_data["group_id"] = training_data["race_id"].astype("category").cat.codes

In [8]:
# Sort by race_id for consistency
training_data = training_data.sort_values("group_id", ascending=True)
training_data.reset_index(drop=True, inplace=True)

## Drop Non-numeric Features

In [9]:
unused_columns = [
    # columns you do NOT use in features or group_id
    "race_date", "date_of_birth"
]
cols_to_drop = [col for col in unused_columns if col in training_data.columns]

training_data.drop(columns=cols_to_drop, inplace=True)
print("After dropping unused cols, shape:", training_data.shape)

After dropping unused cols, shape: (394322, 78)


## Convert DataTime columns to Numerical Values

In [10]:
# Convert datetime columns to numerical
training_data["first_race_date_5"] = pd.to_datetime(training_data["first_race_date_5"])
training_data["most_recent_race_5"] = pd.to_datetime(training_data["most_recent_race_5"])
training_data["prev_race_date"] = pd.to_datetime(training_data["prev_race_date"])

# Calculate numeric date features
training_data["first_race_date_5_numeric"] = (training_data["first_race_date_5"] - pd.Timestamp("1970-01-01")).dt.days
training_data["most_recent_race_5_numeric"] = (training_data["most_recent_race_5"] - pd.Timestamp("1970-01-01")).dt.days
training_data["prev_race_date_numeric"] = (training_data["prev_race_date"] - pd.Timestamp("1970-01-01")).dt.days

# Drop original datetime columns
training_data.drop(columns=["first_race_date_5", "most_recent_race_5", "prev_race_date"], inplace=True)


## Set Rank/Label¶

In [11]:
# If the largest official_fin is 20 (some races can have 20 horses),
# then label = (21 - official_fin).
# So official_fin=1 => label=20, official_fin=2 =>19, etc.
# If your max is 14, you can do (15 - official_fin).  Just ensure "best" horse has largest label.
if "official_fin" in training_data.columns:
    # Calculate 'rank' and add it to the DataFrame
    training_data["rank"] = 21 - training_data["official_fin"]
    # Drop the 'official_fin' column
    training_data.drop(columns=["official_fin"], inplace=True)

## Simple Target Encoding for (XGBoost/LightGBM)

In [12]:
# MAKE SURE YOU KEEP A COPY OF THE original horse_id before encoding.
training_data["horse_id_original"] = training_data["horse_id"]

def target_encode_horse_id(df, horse_col, target_col, n_splits=5):
    """Perform K-fold target encoding for a high-cardinality column (horse_id)."""
    # Make a copy to avoid side-effects
    df_encoded = df.copy()
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    global_mean = df[target_col].mean()  # global average (fallback)
    
    df_encoded["horse_id_encoded"] = np.nan
    
    for train_idx, valid_idx in kf.split(df):
        train_fold = df.iloc[train_idx]
        valid_fold = df.iloc[valid_idx]

        # Compute the mean rank (or finishing) per horse in the train fold
        means = train_fold.groupby(horse_col)[target_col].mean()
        
        # Map onto the validation fold
        df_encoded.loc[valid_fold.index, "horse_id_encoded"] = \
            valid_fold[horse_col].map(means)
    
    # Fill any horse IDs not seen in training folds with the global mean
    df_encoded["horse_id_encoded"] = df_encoded["horse_id_encoded"].fillna(global_mean)
   
    return df_encoded

# Example usage:
training_data = target_encode_horse_id(
    df=training_data,
    horse_col="horse_id",
    target_col="rank",  # or official_finish, or a 0/1 for finishing top 3
    n_splits=5
)

# Now you have "horse_id_encoded" as a numeric column for training.
# You can drop the original horse_id (or keep it if you want to do something else later).
training_data.drop(columns=["horse_id"], inplace=True)

# Assigned Numerical Features

In [13]:

features = ['race_number','horse_id_encoded','purse','weight','claimprice','power','morn_odds','avgspd','class_rating',
    'net_sentiment','avg_spd_sd','ave_cl_sd','hi_spd_sd','pstyerl','all_starts','all_win','all_place',
    'all_show','all_fourth','all_earnings','cond_starts','cond_win','cond_place','cond_show','cond_fourth',
    'cond_earnings','avg_speed_5','best_speed','avg_beaten_len_5','avg_dist_bk_gate1_5','avg_dist_bk_gate2_5',
    'avg_dist_bk_gate3_5','avg_dist_bk_gate4_5','avg_speed_fullrace_5','avg_stride_length_5','avg_strfreq_q1_5',
    'avg_strfreq_q2_5','avg_strfreq_q3_5','avg_strfreq_q4_5','prev_speed','speed_improvement','days_off',
    'avg_workout_rank_3','jock_win_percent','jock_itm_percent','trainer_win_percent','trainer_itm_percent',
    'jt_win_percent','jt_itm_percent','jock_win_track','jock_itm_track','trainer_win_track','trainer_itm_track',
    'jt_win_track','jt_itm_track','age_at_race_day','distance_meters', 'count_workouts_3']

## Set the Category Columns with Label Encoder

In [14]:
# Identify categorical columns
cat_cols = ["course_cd", "sex", "equip", "surface", "med", 
            "race_type", "stk_clm_md", "turf_mud_mark", "layoff_cat"]

# Convert categorical columns
for col in cat_cols:
    if col in training_data.columns:
        training_data[col] = training_data[col].astype("category")

    
# Specify categorical feature indices
cat_cols_indices = [training_data.columns.get_loc(col) for col in cat_cols]

In [15]:
# # Assuming X_predict is your DataFrame

# # Set the display option to show all columns
# pd.set_option('display.max_columns', 100)
# pd.set_option('display.max_info_columns', 100)

# # Display the data types of all columns
# print(training_data.dtypes)

# # Alternatively, use the info method to get a detailed summary
# training_data.info(verbose=True)

# Split Data Into Training and Validation

In [35]:
import pandas as pd
import numpy as np
import xgboost as xgb

# Assuming training_data is your original DataFrame

# Generate unique numeric group_id from race_id
training_data["group_id"] = training_data["race_id"].astype("category").cat.codes

# Split the original DataFrame into training and validation sets
train_races = set(race_id_train)
valid_races = set(race_id_valid)

df_train = training_data[training_data["group_id"].isin(train_races)].reset_index(drop=True)
df_valid = training_data[training_data["group_id"].isin(valid_races)].reset_index(drop=True)

# Combine training and validation datasets for final training
X_full = np.concatenate([X_train, X_valid], axis=0)
y_full = np.concatenate([y_train, y_valid], axis=0)
group_full = np.concatenate([group_train, group_valid], axis=0)

# Create the full DMatrix
full_dmatrix = xgb.DMatrix(X_full, label=y_full, enable_categorical=True)
full_dmatrix.set_group(group_full)

# Predict
preds = xgbm_final.predict(full_dmatrix)

# Build an analysis DataFrame
df_full = pd.concat([df_train, df_valid], axis=0).reset_index(drop=True)
df_analysis = df_full.copy()
df_analysis["pred_score"] = preds
df_analysis["label"] = y_full   # if you want to store the label, not mandatory

# Derive predicted rank
df_analysis = df_analysis.sort_values(by=["race_id", "pred_score"], ascending=[True, False])
df_analysis["pred_rank"] = df_analysis.groupby("race_id").cumcount() + 1

# Assume "official_fin" is a 1-based finishing position => 1 means first place
# df_analysis["actual_rank"] = df_analysis["official_fin"]

In [34]:
df_analysis.columns

Index(['course_cd', 'race_number', 'saddle_cloth_number', 'horse_name',
       'purse', 'weight', 'sex', 'equip', 'claimprice', 'surface', 'power',
       'med', 'morn_odds', 'avgspd', 'race_type', 'class_rating',
       'net_sentiment', 'stk_clm_md', 'turf_mud_mark', 'avg_spd_sd',
       'ave_cl_sd', 'hi_spd_sd', 'pstyerl', 'all_starts', 'all_win',
       'all_place', 'all_show', 'all_fourth', 'all_earnings', 'cond_starts',
       'cond_win', 'cond_place', 'cond_show', 'cond_fourth', 'cond_earnings',
       'total_races_5', 'avg_fin_5', 'avg_speed_5', 'best_speed',
       'avg_beaten_len_5', 'avg_dist_bk_gate1_5', 'avg_dist_bk_gate2_5',
       'avg_dist_bk_gate3_5', 'avg_dist_bk_gate4_5', 'avg_speed_fullrace_5',
       'avg_stride_length_5', 'avg_strfreq_q1_5', 'avg_strfreq_q2_5',
       'avg_strfreq_q3_5', 'avg_strfreq_q4_5', 'prev_speed',
       'speed_improvement', 'days_off', 'layoff_cat', 'avg_workout_rank_3',
       'count_workouts_3', 'jock_win_percent', 'jock_itm_percent',
   

In [38]:


# Example function: top-4 correctness ignoring order
def top4_correctness(group):
    predicted_top4 = set(group.loc[group["pred_rank"] <= 4, "horse_id_encoded"])
    actual_top4    = set(group.loc[group["rank"] <= 4, "horse_id_encoded"])
    correct = len(predicted_top4 & actual_top4)
    return pd.Series({"top4_correct_count": correct})

results_top4 = df_analysis.groupby("race_id").apply(top4_correctness)
counts = results_top4["top4_correct_count"].value_counts()
print("How many correct out of 4 (ignoring order)?")
print(counts)

# Example function: exact top-4 in order
def exact_top4_positions(group):
    actual_sorted = group.sort_values("rank")["horse_id_encoded"].tolist()
    pred_sorted   = group.sort_values("pred_rank")["horse_id_encoded"].tolist()

    # just take the first 4
    actual_top4 = actual_sorted[:4]
    pred_top4   = pred_sorted[:4]

    exact_positions = sum(a == b for a, b in zip(actual_top4, pred_top4))
    return pd.Series({"exact_positions_0to4": exact_positions})

results_order = df_analysis.groupby("race_id").apply(exact_top4_positions)
dist = results_order["exact_positions_0to4"].value_counts()
print("Exact top-4 order distribution (0..4):")
print(dist)

/tmp/ipykernel_3782359/552967971.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results_top4 = df_analysis.groupby("race_id").apply(top4_correctness)


How many correct out of 4 (ignoring order)?
top4_correct_count
0    51075
Name: count, dtype: int64
Exact top-4 order distribution (0..4):
exact_positions_0to4
0    34179
1    14776
2     1946
3      162
4       12
Name: count, dtype: int64


/tmp/ipykernel_3782359/552967971.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results_order = df_analysis.groupby("race_id").apply(exact_top4_positions)


In [39]:
34179/51075

0.6691923641703378

In [17]:
X_train, y_train, race_id_train, X_valid, y_valid, race_id_valid = split_train_valid(
    training_data=training_data,           # The entire DF
    race_id_col="group_id",           # The column name for group/race
    features=features,                # List of feature columns
    target="rank",                    # Name of target column
    train_ratio=0.8,
    random_state=42
)

# Define Group Array

In [18]:
# Generate group sizes
def make_group_array(race_id_array):
    groups = []
    current_race = None
    current_count = 0

    for rid in race_id_array:
        if rid != current_race:
            if current_race is not None:
                groups.append(current_count)
            current_race = rid
            current_count = 1
        else:
            current_count += 1
    if current_race is not None and current_count > 0:
        groups.append(current_count)
    
    return np.array(groups, dtype=np.int32)

group_train = make_group_array(race_id_train)
group_valid = make_group_array(race_id_valid)

# Ensure correct group sizes
assert sum(group_train) == len(X_train), "Mismatch between group sizes and training data."
assert sum(group_valid) == len(X_valid), "Mismatch between group sizes and validation data."


# Make DMatrix Object

In [19]:

# Create DMatrices with group
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features, enable_categorical=True)
dtrain.set_group(group_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid, feature_names=features, enable_categorical=True)
dvalid.set_group(group_valid)

# Ensure correct group sizes
assert sum(group_train) == len(X_train), "Mismatch between group sizes and training data."
assert sum(group_valid) == len(X_valid), "Mismatch between group sizes and validation data."

# XGBoost
(Use Optuna Hyperparameter tuning below)

In [20]:
best_score = float("-inf")
best_params = None
best_iteration = None

# Hyperparameter Tuning with Optuna

In [22]:
import optuna
import xgboost as xgb

def objective(trial):
    # 1) Suggest XGBoost hyperparams
    params = {
        # Core ranking objective and eval
        "booster": "gbtree",
        "objective": "rank:ndcg",            # or "rank:pairwise"
        "eval_metric": "ndcg@4",
        # GPU usage in XGBoost 2.0+
        "tree_method": "hist",               # "hist" + "device":"cuda"
        "device": "cuda",
        
        # 2) Hyperparams you tune
        "eta": trial.suggest_float("eta", 0.001, 0.5, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 15),  # expanded range
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 100),  # expanded range
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),    # expanded range
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),  # expanded range

        # 3) Regularization
        "alpha": trial.suggest_float("alpha", 0.0, 20.0),   # expanded range
        "lambda": trial.suggest_float("lambda", 0.0, 20.0), # expanded range
        "gamma": trial.suggest_float("gamma", 0.0, 10.0),    # expanded range

        # 4) Hist bin
        "max_bin": trial.suggest_int("max_bin", 32, 512),   # GPU-compatible

        # 5) Additional hyperparameters
        "max_delta_step": trial.suggest_float("max_delta_step", 0.0, 10.0)
    }

    # 5) Create DMatrices with group
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtrain.set_group(group_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
    dvalid.set_group(group_valid)

    watchlist = [(dtrain, "train"), (dvalid, "valid")]

    # 6) Train
    xgb_model = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=1000,
        evals=watchlist,
        early_stopping_rounds=50,
        verbose_eval=50
    )

    # 7) The best validation score is directly xgb_model.best_score, 
    # if "valid" is the second item in watchlist and you have one eval_metric
    best_score_ndcg4 = xgb_model.best_score
    
    return best_score_ndcg4

# Run Optuna optimization
def run_optuna():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=200)  # Increased number of trials for better exploration
    return study

study = run_optuna()
best_params = study.best_params
best_value = study.best_value   # Best ndcg@4
best_ndcg4 = best_value

print("Best params:", best_params)
print("Best ndcg@4:", best_ndcg4)

[I 2025-01-13 16:45:57,409] A new study created in memory with name: no-name-d6a78a85-1131-4b34-8a8d-2e59cefe0495


[0]	train-ndcg@4:0.79018	valid-ndcg@4:0.78881
[50]	train-ndcg@4:0.83034	valid-ndcg@4:0.82791
[100]	train-ndcg@4:0.83550	valid-ndcg@4:0.83303
[150]	train-ndcg@4:0.83716	valid-ndcg@4:0.83481
[200]	train-ndcg@4:0.83923	valid-ndcg@4:0.83643
[250]	train-ndcg@4:0.84117	valid-ndcg@4:0.83802
[300]	train-ndcg@4:0.84182	valid-ndcg@4:0.83852
[350]	train-ndcg@4:0.84269	valid-ndcg@4:0.83932
[400]	train-ndcg@4:0.84414	valid-ndcg@4:0.84081
[450]	train-ndcg@4:0.84616	valid-ndcg@4:0.84292
[500]	train-ndcg@4:0.84674	valid-ndcg@4:0.84332
[550]	train-ndcg@4:0.84786	valid-ndcg@4:0.84440
[600]	train-ndcg@4:0.84903	valid-ndcg@4:0.84535
[650]	train-ndcg@4:0.85024	valid-ndcg@4:0.84641
[700]	train-ndcg@4:0.85132	valid-ndcg@4:0.84735
[750]	train-ndcg@4:0.85288	valid-ndcg@4:0.84885
[800]	train-ndcg@4:0.85388	valid-ndcg@4:0.85002
[850]	train-ndcg@4:0.85502	valid-ndcg@4:0.85134
[900]	train-ndcg@4:0.85624	valid-ndcg@4:0.85265
[950]	train-ndcg@4:0.85747	valid-ndcg@4:0.85348
[999]	train-ndcg@4:0.85849	valid-ndcg@4:0.8

[I 2025-01-13 16:46:29,750] Trial 0 finished with value: 0.8546916753612182 and parameters: {'eta': 0.0023487431935336218, 'max_depth': 8, 'min_child_weight': 37, 'subsample': 0.5245024476670422, 'colsample_bytree': 0.652494534551351, 'alpha': 13.431669780421924, 'lambda': 18.024863522833453, 'gamma': 4.202278007038645, 'max_bin': 107, 'max_delta_step': 1.4968329280907677}. Best is trial 0 with value: 0.8546916753612182.


[0]	train-ndcg@4:0.69977	valid-ndcg@4:0.69567
[50]	train-ndcg@4:0.86248	valid-ndcg@4:0.85749
[100]	train-ndcg@4:0.87887	valid-ndcg@4:0.87258
[150]	train-ndcg@4:0.88748	valid-ndcg@4:0.88132
[200]	train-ndcg@4:0.89476	valid-ndcg@4:0.88798
[250]	train-ndcg@4:0.89919	valid-ndcg@4:0.89229
[300]	train-ndcg@4:0.90333	valid-ndcg@4:0.89637
[350]	train-ndcg@4:0.90558	valid-ndcg@4:0.89818
[400]	train-ndcg@4:0.90707	valid-ndcg@4:0.89944
[450]	train-ndcg@4:0.90882	valid-ndcg@4:0.90119
[500]	train-ndcg@4:0.90962	valid-ndcg@4:0.90168
[550]	train-ndcg@4:0.91055	valid-ndcg@4:0.90263
[600]	train-ndcg@4:0.91130	valid-ndcg@4:0.90319
[650]	train-ndcg@4:0.91216	valid-ndcg@4:0.90443
[700]	train-ndcg@4:0.91250	valid-ndcg@4:0.90482
[750]	train-ndcg@4:0.91326	valid-ndcg@4:0.90538
[800]	train-ndcg@4:0.91376	valid-ndcg@4:0.90603
[850]	train-ndcg@4:0.91425	valid-ndcg@4:0.90661
[900]	train-ndcg@4:0.91454	valid-ndcg@4:0.90685
[950]	train-ndcg@4:0.91497	valid-ndcg@4:0.90741
[999]	train-ndcg@4:0.91518	valid-ndcg@4:0.9

[I 2025-01-13 16:47:00,347] Trial 1 finished with value: 0.9077363815369354 and parameters: {'eta': 0.08266154423756025, 'max_depth': 8, 'min_child_weight': 3, 'subsample': 0.6177135692941399, 'colsample_bytree': 0.4477494572711738, 'alpha': 14.466005333669374, 'lambda': 16.723194226142027, 'gamma': 1.9868921770842807, 'max_bin': 215, 'max_delta_step': 8.511508681840748}. Best is trial 1 with value: 0.9077363815369354.


...

[0]	train-ndcg@4:0.78785	valid-ndcg@4:0.78684
[50]	train-ndcg@4:0.87582	valid-ndcg@4:0.87303
[100]	train-ndcg@4:0.89625	valid-ndcg@4:0.89124
[150]	train-ndcg@4:0.90616	valid-ndcg@4:0.89993
[200]	train-ndcg@4:0.91224	valid-ndcg@4:0.90427
[250]	train-ndcg@4:0.91658	valid-ndcg@4:0.90684
[300]	train-ndcg@4:0.91932	valid-ndcg@4:0.90901
[350]	train-ndcg@4:0.92165	valid-ndcg@4:0.91039
[400]	train-ndcg@4:0.92333	valid-ndcg@4:0.91113
[450]	train-ndcg@4:0.92486	valid-ndcg@4:0.91207
[500]	train-ndcg@4:0.92608	valid-ndcg@4:0.91277
[550]	train-ndcg@4:0.92702	valid-ndcg@4:0.91302
[600]	train-ndcg@4:0.92784	valid-ndcg@4:0.91325
[650]	train-ndcg@4:0.92880	valid-ndcg@4:0.91378
[700]	train-ndcg@4:0.92934	valid-ndcg@4:0.91378
[750]	train-ndcg@4:0.92983	valid-ndcg@4:0.91431
[800]	train-ndcg@4:0.93031	valid-ndcg@4:0.91465
[850]	train-ndcg@4:0.93072	valid-ndcg@4:0.91469
[877]	train-ndcg@4:0.93091	valid-ndcg@4:0.91475


[I 2025-01-13 18:14:18,255] Trial 198 finished with value: 0.9147921727570627 and parameters: {'eta': 0.10423890295140234, 'max_depth': 6, 'min_child_weight': 14, 'subsample': 0.8344566723445908, 'colsample_bytree': 0.9746517939637467, 'alpha': 4.5548377459135025, 'lambda': 9.056916514668467, 'gamma': 0.9595106844821419, 'max_bin': 282, 'max_delta_step': 8.127922537563101}. Best is trial 198 with value: 0.9147921727570627.


[0]	train-ndcg@4:0.78848	valid-ndcg@4:0.78716
[50]	train-ndcg@4:0.87685	valid-ndcg@4:0.87317
[100]	train-ndcg@4:0.89682	valid-ndcg@4:0.89130
[150]	train-ndcg@4:0.90682	valid-ndcg@4:0.89942
[200]	train-ndcg@4:0.91278	valid-ndcg@4:0.90467
[250]	train-ndcg@4:0.91673	valid-ndcg@4:0.90763
[300]	train-ndcg@4:0.91973	valid-ndcg@4:0.90867
[350]	train-ndcg@4:0.92188	valid-ndcg@4:0.90996
[400]	train-ndcg@4:0.92373	valid-ndcg@4:0.91081
[450]	train-ndcg@4:0.92520	valid-ndcg@4:0.91186
[500]	train-ndcg@4:0.92633	valid-ndcg@4:0.91229
[550]	train-ndcg@4:0.92738	valid-ndcg@4:0.91278
[600]	train-ndcg@4:0.92832	valid-ndcg@4:0.91295
[650]	train-ndcg@4:0.92910	valid-ndcg@4:0.91350
[700]	train-ndcg@4:0.92978	valid-ndcg@4:0.91412
[750]	train-ndcg@4:0.93046	valid-ndcg@4:0.91413
[800]	train-ndcg@4:0.93088	valid-ndcg@4:0.91423
[817]	train-ndcg@4:0.93099	valid-ndcg@4:0.91420


[I 2025-01-13 18:14:44,007] Trial 199 finished with value: 0.9143468986591095 and parameters: {'eta': 0.10542755149160091, 'max_depth': 6, 'min_child_weight': 15, 'subsample': 0.8342736304601023, 'colsample_bytree': 0.9569249445475796, 'alpha': 4.585571584838654, 'lambda': 9.112202163727593, 'gamma': 0.9123479618793059, 'max_bin': 253, 'max_delta_step': 8.181294850522494}. Best is trial 198 with value: 0.9147921727570627.


Best params: {'eta': 0.10423890295140234, 'max_depth': 6, 'min_child_weight': 14, 'subsample': 0.8344566723445908, 'colsample_bytree': 0.9746517939637467, 'alpha': 4.5548377459135025, 'lambda': 9.056916514668467, 'gamma': 0.9595106844821419, 'max_bin': 282, 'max_delta_step': 8.127922537563101}
Best ndcg@4: 0.9147921727570627


# Final XGB Run with Optuna Parameters

In [ ]:
print("Best params:", best_params)
print("Best ndcg@4:", best_ndcg4)

In [23]:
# Best parameters from Optuna tuning for XGB
final_params = {
        # Core ranking objective and eval
        "booster": "gbtree",
        "objective": "rank:ndcg",            # or "rank:pairwise"
        "eval_metric": "ndcg@4",
        # GPU usage in XGBoost 2.0+
        "tree_method": "hist",               # "hist" + "device":"cuda"
        "device": "cuda",
        
        # 2) Hyperparams you tune
        "eta": 0.10423890295140234,
        "max_depth": 6,  # typical XGBoost range
        "min_child_weight": 14,  # LGB's min_data_in_leaf
        "subsample": 0.8344566723445908,    # LGB's bagging_fraction
        "colsample_bytree": 0.9746517939637467,  # LGB's feature_fraction

        # 3) Regularization
        "alpha": 4.5548377459135025,   # LGB's lambda_l1
        "lambda": 9.056916514668467, # LGB's lambda_l2
        "gamma": 0.9595106844821419,    # LGB's min_gain_to_split

        # 4) Hist bin
        "max_bin": 282,   # GPU-compatible
        'max_delta_step': 8.127922537563101
    }




In [24]:
# Combine training and validation datasets for final training
X_full = np.concatenate([X_train, X_valid], axis=0)
y_full = np.concatenate([y_train, y_valid], axis=0)
group_full = np.concatenate([group_train, group_valid], axis=0)

# Create the full DMatrix
full_dmatrix = xgb.DMatrix(X_full, label=y_full, enable_categorical=True)
full_dmatrix.set_group(group_full)


In [25]:
# Train the final model (no evals since it's full training)
xgbm_final = xgb.train(
    params=final_params,
    dtrain=full_dmatrix,
    num_boost_round=5000,
    verbose_eval=50
)

# Save the best XGB Model

In [26]:
# 5) Save the final model to disk:
xgbm_final.save_model("/home/exx/myCode/horse-racing/FoxRiverAIRacing/src/models/XGB_91479_2025-01-13.json")

# Predict

In [ ]:
preds = xgbm_final.predict(full_dmatrix)

In [ ]:
training_data.shape[0] == X_full.shape[0]

In [ ]:
df_analysis = training_data.copy()  # or create new from scratch
df_analysis["pred_score"] = preds  # attach predicted ranking scores
df_analysis["true_label"] = y_full # attach the label (some measure of actual rank or finishing position)

In [ ]:
df_analysis[["rank", "true_label"]].head(100)

In [ ]:
df_analysis = df_analysis.sort_values(
    by=["race_id", "pred_score"],
    ascending=[True, False]
)

# For each race_id, the row with the highest 'pred_score' becomes predicted_rank=1,
# the second highest is predicted_rank=2, etc.

df_analysis["pred_rank"] = (
    df_analysis.groupby("race_id").cumcount() + 1
)

In [ ]:
# A common approach is:
#   official_fin = 1 means 1st place, official_fin=2 means 2nd place, etc.
#   So we can define "actual_rank" simply as official_fin itself (if 1=first).
#   Or we can do a grouping sort if your y_full is not exactly that.

df_analysis["actual_rank"] = df_analysis["rank"]  # if it already is 1=first, 2=second, ...

# Alternatively, if your label was "21 - official_fin", you might prefer re-sorting
# to derive an actual finishing order. For example:
#   (lowest official_fin => actual_rank=1)
#   Or if you only stored the label, you'd do something else.

In [ ]:
def top4_accuracy_per_race(group: pd.DataFrame):
    # group is the subset of rows for one race
    # pick the predicted top‐4
    predicted_top4 = set(group.loc[group["pred_rank"] <= 4, "horse_id"])
    # pick the actual top‐4
    actual_top4    = set(group.loc[group["official_fin"] <= 4, "horse_id"])
    # intersection size
    correct_in_top4 = len(predicted_top4 & actual_top4)
    return pd.Series({
        "correct_in_top4": correct_in_top4,  # e.g. 0..4
        "size_pred_top4":  len(predicted_top4),
        "size_true_top4":  len(actual_top4),
    })

results = df_analysis.groupby("race_id").apply(top4_accuracy_per_race)
results.head()

In [ ]:
counts = results["correct_in_top4"].value_counts()
counts_percentage = counts / len(results) * 100.0
print("Distribution of how many correct out of the top‐4:")
print(counts_percentage)

In [ ]:
def exact_top4_order_stats(group):
    # Sort rows by actual_rank ascending => actual top order
    actual_sorted = group.sort_values("official_fin")["horse_id"].tolist()
    actual_top4   = actual_sorted[:4]  # first 4 entries are the top‐4

    # Sort rows by predicted rank => predicted top order
    pred_sorted = group.sort_values("pred_rank")["horse_id"].tolist()
    pred_top4   = pred_sorted[:4]

    # measure how many in the same position
    exact_matches = sum(a == b for (a,b) in zip(actual_top4, pred_top4))
    return pd.Series({"exact_top4_positions_matched": exact_matches})

df_positions = df_analysis.groupby("race_id").apply(exact_top4_order_stats)
dist = df_positions["exact_top4_positions_matched"].value_counts()
print(dist)

In [ ]:
# If you stored official_fin in df_full, you can keep it:
df_analysis["official_fin"] = df_full["official_fin"]

In [32]:
# Suppose your original DataFrame is `df_all` with shape (N, many_columns)
# and you identify a set of features to pass to XGB (feature_cols),
# plus a label column ("rank"):

feature_cols = [...]
label_col = "rank"

# 1) Create X_full, y_full (array forms).
X_full = df_all[feature_cols].values     # shape (N, d)
y_full = df_all[label_col].values        # shape (N,)

# 2) Build + train your model
#    e.g. xgbm_final = xgb.train(...) or xgb_model.fit(X_full, y_full) ...
#    (Assume we do that and then produce predictions:)
preds = xgbm_final.predict(xgb.DMatrix(X_full))

# 3) Attach predictions back to the original df_all
df_all["pred_score"] = preds

# 4) If you want a separate "analysis" DataFrame, copy df_all AFTER you attach:
df_analysis = df_all.copy()

# Now do your rank logic or measure top-4 accuracy, etc.
# e.g. sort within each race by pred_score descending, etc.

NameError: name 'df_all' is not defined

In [ ]:
df_analysis = df_full.copy()  # or create new from scratch
df_analysis["pred_score"] = preds  # attach predicted ranking scores
df_analysis["true_label"] = y_full # attach the label (some measure of actual rank or finishing position)

In [ ]:
y_pred = model.predict(X_eval)  # e.g. XGB, LGB, Cat
df_eval["pred_score"] = y_pred

# Compute predicted rank by sorting by pred_score descending, within each race_id.

In [ ]:
df_eval = df_eval.sort_values(by=["race_id", "pred_score"], ascending=[True, False])
df_eval["pred_rank"] = df_eval.groupby("race_id").cumcount() + 1A

In [ ]:
def analyze_top4_vs_actual(df):
    """
    df: must contain columns race_id, official_fin, pred_rank
    returns a DataFrame with counts or distribution per race
    or a single summary
    """

    # Example: exact order correctness
    def check_exact_top4(group):
        actual_top4 = (group.sort_values("official_fin").head(4)["horse_id"].tolist())
        pred_top4   = (group.sort_values("pred_rank").head(4)["horse_id"].tolist())

        exact_count = sum(a == b for a, b in zip(actual_top4, pred_top4))
        return pd.Series({"exact_4_matches": (1 if exact_count == 4 else 0),
                          "exact_count": exact_count})

    results = df.groupby("race_id").apply(check_exact_top4).reset_index()
    # e.g. compute distribution:
    dist_exact_count = results["exact_count"].value_counts()
    # or the fraction:
    fraction_exact4  = results["exact_4_matches"].mean()

    # Return a dictionary or print it
    return {
        "exact_count_distribution": dist_exact_count.to_dict(),
        "fraction_4of4_exact": fraction_exact4
    }

In [ ]:
# Get feature importance scores
importance = xgbm_final.get_score(importance_type='weight')

# Convert to a DataFrame for easier manipulation
importance_df = pd.DataFrame({
    'Feature': [features[int(f[1:])] for f in importance.keys()],
    'Importance': list(importance.values())
})

# Sort by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print the feature importance scores
print(importance_df)

In [ ]:
# 6) Predict on validation set or a new test set
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming you have a trained XGBoost model named `xgb_model`

# Get feature importance scores
importance = xgbm_final.get_score(importance_type='weight')

# Convert to a DataFrame for easier plotting
importance_df = pd.DataFrame({
    'Feature': list(importance.keys()),
    'Importance': list(importance.values())
})

# Sort by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Feature Importance')
plt.show()

In [ ]:
print(preds)